<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.26.0
Pandas:  2.1.0


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

#They are wronng due to the use of incorrect file path.

FileNotFoundError: [Errno 2] No such file or directory: 'bikeshare.csv'

ANSWER:

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [4]:
#ANSWER:
bikes = pd.read_table('../../DATA/bikeshare.csv', header = 0, sep=',')
bikes

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be dealt with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [5]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [6]:
df = pd.read_excel('../../DATA/Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For example, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [7]:
#ANSWER
df = pd.read_excel('../../DATA/Iris.xls', sheet_name = 'Data', header=0, usecols=[0, 1, 2, 3, 4])
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [10]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-12-11T06:33:31.043Z,-18.6761,-175.4826,238.398,6.10,mww,89,32,2.4950,1.38,...,2023-12-11T06:50:12.040Z,"158 km W of Neiafu, Tonga",earthquake,9.31,5.908,0.063000,24,reviewed,us,us
1,2023-12-11T05:51:25.620Z,19.2080,-64.8765,14.370,3.32,md,13,277,0.8204,0.18,...,2023-12-11T06:16:21.830Z,"96 km N of Charlotte Amalie, U.S. Virgin Islands",earthquake,2.97,5.670,0.069063,5,reviewed,pr,pr


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [12]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [13]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here are a few examples: https://pandas.pydata.org/docs/user_guide/io.html#io-read-xml

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example:

In [14]:
url = 'https://microsoftedge.github.io/Demos/json-dummy-data/64KB.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,name,language,id,bio,version
0,Adeel Solangi,Sindhi,V59OF92YF627HFY0,Donec lobortis eleifend condimentum. Cras dict...,6.10
1,Afzal Ghaffar,Sindhi,ENTOCR13RSCLZ6KU,"Aliquam sollicitudin ante ligula, eget malesua...",1.88
2,Aamir Solangi,Sindhi,IAKPO3R4761JDRVG,Vestibulum pharetra libero et velit gravida eu...,7.27
3,Abla Dilmurat,Uyghur,5ZVOEPMJUI4MB4EN,Donec lobortis eleifend condimentum. Morbi ac ...,2.53
4,Adil Eli,Uyghur,6VTI8X6LL0MMPJCC,"Vivamus id faucibus velit, id posuere leo. Mor...",6.49


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [17]:
#ANSWER
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [18]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [19]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [20]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this.
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [38]:
#ANSWER
print(f"object type of bikes['season'] is {type(bikes['season'])}\n\
object type of bikes[['season']] is {type(bikes[['season']])}\n\n\
bikes['season'] returns a Series, whilst bikes[['season']] returns a DataFrame")

object type of bikes['season'] is <class 'pandas.core.series.Series'>
object type of bikes[['season']] is <class 'pandas.core.frame.DataFrame'>

bikes['season'] returns a Series, whilst bikes[['season']] returns a DataFrame


How would we use object notation to show the first 4 rows of `atemp`?

In [41]:
#ANSWER
bikes.atemp[:4]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ):

In [42]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Index([0], dtype='int32')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [44]:
#ANSWER
bikes.iloc[0, :]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed            0.0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [45]:
#ANSWER
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for?

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [46]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [52]:
#ANSWER
bikes.isna().sum().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [53]:
#ANSWER
bikes.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

##### np.nan is used to present null (missing) values

?
ANSWER:

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [56]:
#ANSWER
def div(dividend, divisor):
    if divisor == 0:
        outcome = np.nan
    else:
        outcome = dividend / divisor
    return outcome

div(4,0)

nan

Apply the Pandas `isna` function to the following data objects:

In [57]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [59]:
#ANSWER
y.isna()

AttributeError: 'float' object has no attribute 'isna'

In [61]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [64]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER:

The na_values parameter in pandas I/O functions is used to specify additional values that should be treated as NaN| during the reading process.

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [67]:
#ANSWER
bikes['season'].unique()

array([1, 2, 3, 4], dtype=int64)

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns:

In [80]:
#ANSWER
print(f"Range of bikes['instant']: {bikes['instant'].min()}, {bikes['instant'].max()}\n\
Range of bikes['dteday']: {bikes['dteday'].min()}, {bikes['dteday'].max()}\n\
Range of bikes['windspeed']: {bikes['windspeed'].min()}, {bikes['windspeed'].max()}")

Range of bikes['instant']: 1, 17379
Range of bikes['dteday']: 2011-01-01, 2012-12-31
Range of bikes['windspeed']: 0.0, 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [83]:
#Select columns with numeric data
bikes_numeric = bikes.select_dtypes(include='number')
bikes_numeric

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


In [84]:
bikes_min, bikes_max = (min(bikes_numeric.min()), max(bikes_numeric.max()))
bikes_min, bikes_max

(0.0, 17379.0)

In [85]:
#OR
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

(0.0, 17379.0)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [91]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code:

In [92]:
#ANSWER
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [93]:
type(bikes['atemp'])

pandas.core.series.Series

In [94]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
13399,13400,2012-07-17,3,1,7,12,0,2,1,2,0.90,0.8182,0.35,0.0000,80,174,254
8504,8505,2011-12-26,1,0,12,1,1,1,0,1,0.28,0.2727,0.56,0.1940,3,7,10
1549,1550,2011-03-09,1,0,3,22,0,3,1,3,0.32,0.3182,0.93,0.1642,4,17,21
12749,12750,2012-06-20,2,1,6,10,0,3,1,1,0.82,0.7727,0.52,0.1343,41,126,167
6753,6754,2011-10-13,4,0,10,23,0,4,1,2,0.58,0.5455,0.88,0.1940,2,45,47


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indicates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [96]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)
atemp_level

0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

What is `atemp_level`?

In [97]:
#ANSWER
atemp_level

0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

Here is a random sample of `atemp_level`:

In [98]:
atemp_level.sample(5)

3234     (0.5, 0.75]
6388     (0.5, 0.75]
15666    (0.25, 0.5]
16986    (0.25, 0.5]
15075    (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [99]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)

5705     warm
11110    mild
4081     warm
12767    warm
10376    warm
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [101]:
#ANSWER
bikes['atemp_level'] = pd.cut(bikes['atemp'], bins=4, labels=['cool', 'mild', 'warm', 'hot'])
bikes.loc[(bikes['season'] == 2) & (bikes['atemp_level'] == 'mild'), 'atemp_level'].count()

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initialise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [104]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

## Load Data

Load rock.csv and clean the dataset.

In [223]:
# ANSWER
df = pd.read_csv('../../DATA/rock.csv')
df

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


## Check Column Names

Check column names and clean.

In [224]:
# ANSWER
# Extract column names and put them into a list
column_name = df.columns

# Convert to lower case
column_name = column_name.str.lower()
# Remove leading and trailing spaces
column_name = column_name.str.strip()
# Replace white spaces with underscore
column_name = column_name.str.replace(' ', '_')
# Remove '?'
column_name = column_name.str.replace('?', '')
# Rename 'F*G' column to 'Product_of_year_and_playCount'
df.columns = column_name
df.rename(columns={'f*g': 'product_of_year_and_playcount'}, inplace=True)
df.columns

Index(['song_clean', 'artist_clean', 'release_year', 'combined', 'first',
       'year', 'playcount', 'product_of_year_and_playcount'],
      dtype='object')

## Replace Null Values With 0

Check 'release_year' column whether this column has any null values or not. Replace null values with 0.

In [225]:
# ANSWER
# Check for null values
df['release_year'].isna().sum()
# Replace null with 0
df['release_year'].fillna(value=0, inplace=True)
df['release_year'].isna().sum()

0

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column.

In [226]:
# ANSWER
# Check data types
print(df.dtypes)

# Check values in 'release_year'
print(df['release_year'].unique())

# Replace 'songfacts.com' with 0
df.loc[df['release_year'] == 'SONGFACTS.COM', 'release_year'] = 0
df['release_year'] = df['release_year'].astype('int64')
df.dtypes

song_clean                       object
artist_clean                     object
release_year                     object
combined                         object
first                             int64
year                              int64
playcount                         int64
product_of_year_and_playcount     int64
dtype: object
['1982' 0 '1981' '1980' '1975' '2000' '2002' '1992' '1985' '1993' '1976'
 '1995' '1979' '1984' '1977' '1990' '1986' '1974' '2014' '1987' '1973'
 '2001' '1989' '1997' '1971' '1972' '1994' '1970' '1966' '1965' '1983'
 '1955' '1978' '1969' '1999' '1968' '1988' '1962' '2007' '1967' '1958'
 '1071' '1996' '1991' '2005' '2011' '2004' '2012' '2003' '1998' '2008'
 '1964' '2013' '2006' 'SONGFACTS.COM' '1963' '1961']


song_clean                       object
artist_clean                     object
release_year                      int64
combined                         object
first                             int64
year                              int64
playcount                         int64
product_of_year_and_playcount     int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [227]:
# ANSWER
df.describe()

,release_year,first,year,playcount,product_of_year_and_playcount
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1465.331390,1.0,0.741256,16.872646,15.048430
std,867.196161,0.0,0.438043,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000


In [228]:
df.loc[df['release_year'] != 0].min()

song_clean                                      #9 Dream
artist_clean                                 .38 Special
release_year                                        1071
combined                         #9 Dream by John Lennon
first                                                  1
year                                                   1
playcount                                              0
product_of_year_and_playcount                          0
dtype: object

We can replace the release_year, '1071' in this case, with a different value. However, in order to do this, further research needs to be done in order to make sure we replace it with an accurate value

## Write Some Functions

### Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [229]:
# ANSWER
def song_artist_yearcheck(dataframe, index):
    if dataframe.loc[index, 'release_year'] >= 1970:
        print(f"""\
        \nSong: {dataframe.loc[index, 'song_clean']}\
        \nArtist: {dataframe.loc[index, 'artist_clean']}\
        \nRelease_year: 1970 onwards\
        """)
    else:
        print(f"""\
        \nSong: {dataframe.loc[index, 'song_clean']}\
        \nArtist: {dataframe.loc[index, 'artist_clean']}\
        \nRelease_year: before 1970\
        """)

In [230]:
song_artist_yearcheck(df, 100)

        
Song: One Way Out        
Artist: Allman Brothers Band        
Release_year: before 1970        


### Write a function that converts a column in a DataFrame to a numeric type and otherwise replaces entries with np.nan

In [221]:
# ANSWER
def covert_to_numeric(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

### Apply this last function to your dataset

In [240]:
df = pd.read_csv('../../DATA/rock.csv')
df

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


In [241]:
# ANSWER
# Extract column names and put them into a list
column_name = df.columns

# Convert to lower case
column_name = column_name.str.lower()
# Remove leading and trailing spaces
column_name = column_name.str.strip()
# Replace white spaces with underscore
column_name = column_name.str.replace(' ', '_')
# Remove '?'
column_name = column_name.str.replace('?', '')
# Rename 'F*G' column to 'Product_of_year_and_playCount'
df.columns = column_name
df.rename(columns={'f*g': 'product_of_year_and_playcount'}, inplace=True)
df.columns

Index(['song_clean', 'artist_clean', 'release_year', 'combined', 'first',
       'year', 'playcount', 'product_of_year_and_playcount'],
      dtype='object')

In [244]:
# ANSWER
df_new = df[['release_year', 'first', 'year', 'playcount', 'product_of_year_and_playcount']].apply(covert_to_numeric)

### 'Describe' the new DataFrame

In [245]:
# ANSWER
df_new.describe().T

,count,mean,std,min,25%,50%,75%,max
release_year,1652.0,1978.019976,24.191247,1071.0,1971.0,1977.0,1984.0,2014.0
first,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
year,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
playcount,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
product_of_year_and_playcount,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0


In [246]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
first,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
year,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
playcount,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
product_of_year_and_playcount,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0


>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



